In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Day 3: Multi-Layer Perceptrons

In this lab, we aim to solve the problem of classifying human faces into males and females. We have at our disposition a dataset of 200 images of celebrity faces and their associated labels (0 for male and 1 for female).  

We  use a face detector from the dlib library to  estimate  the  location  of  68  (x,  y) coordinates  that  map  to specific facial regions. The image below visualizes what each of these coordinates maps to:


![](face_feature_extraction.png)

We then use tensorflow to define and train a multi-layer perceptron (MLP) graph to classify images using the features visualized above. Using the code below, try to apply the following changes:
    
1 - Change the complexity of the 2-layer MLP by increasing or decreasing the number of neurons in each layer.

2 - Try increasing the number of layers used. This should increase the "depth" of the MLP. To do so, you must change the definition of the MLP function in "multilayer_perceptron" and the weights/biases allocation function "allocate_weights_and_biases".

3 - Try changing how the weights and parameters are initialized. What would happen if you initialize all parameters to zero ?

4 - Try increasing or decreasing the learning rate and number of training epochs. How does this affect the "fitting" to training data ?


### Import APIs to be used 

In [ ]:
from importlib.machinery import SourceFileLoader

somemodule = SourceFileLoader('lab3_data', '/content/drive/MyDrive/Collab/lab7/lab3_data.py').load_module()

In [ ]:
# import tensorflow as tf

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import lab3_data as import_data
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


### Load  CelebA data and create train and test splits (Train: 100 exmaples, Test: 100 examples)

In [ ]:
def get_data():
    X, y = import_data.extract_features_labels()
    Y = np.array([y, -(y - 1)]).T
    tr_X = X[:100] ; tr_Y = Y[:100]
    te_X = X[100:] ; te_Y = Y[100:]

    return tr_X, tr_Y, te_X, te_Y

### Allocate memory for weights and biases for all MLP layers
You can try changing the number of neurons to increase or decrease the complexity of the MLP.

In [ ]:
def allocate_weights_and_biases():

    # define number of hidden layers ..
    n_hidden_1 = 2048  # 1st layer number of neurons
    n_hidden_2 = 2048  # 2nd layer number of neurons

    # inputs placeholders
    X = tf.placeholder("float", [None, 68, 2])
    Y = tf.placeholder("float", [None, 2])  # 2 output classes
    
    # flatten image features into one vector (i.e. reshape image feature matrix into a vector)
    # images_flat = tf.contrib.layers.flatten(X)  
    images_flat = tf.compat.v1.layers.flatten(X)  
    
    
    # weights and biases are initialized from a normal distribution with a specified standard devation stddev
    stddev = 0.01
    
    # define placeholders for weights and biases in the graph
    weights = {
        'hidden_layer1': tf.Variable(tf.random_normal([68 * 2, n_hidden_1], stddev=stddev)),
        'hidden_layer2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
        'out': tf.Variable(tf.random_normal([n_hidden_2, 2], stddev=stddev))
    }

    biases = {
        'bias_layer1': tf.Variable(tf.random_normal([n_hidden_1], stddev=stddev)),
        'bias_layer2': tf.Variable(tf.random_normal([n_hidden_2], stddev=stddev)),
        'out': tf.Variable(tf.random_normal([2], stddev=stddev))
    }
    
    return weights, biases, X, Y, images_flat
    

### Define how the weights and biases are used for inferring classes from inputs (i.e. define MLP function)

You can add more layers to the MLP to fit more complicated functions. Adding more layers requires more learnable weights and biases, which need to defined in "allocate_weights_and_biases" first.

In [ ]:
# Create model
def multilayer_perceptron():
        
    weights, biases, X, Y, images_flat = allocate_weights_and_biases()

    # Hidden fully connected layer 1
    layer_1 = tf.add(tf.matmul(images_flat, weights['hidden_layer1']), biases['bias_layer1'])
    layer_1 = tf.sigmoid(layer_1)

    # Hidden fully connected layer 2
    layer_2 = tf.add(tf.matmul(layer_1, weights['hidden_layer2']), biases['bias_layer2'])
    layer_2 = tf.sigmoid(layer_2)
    
    # Output fully connected layer
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']

    return out_layer, X, Y



### Define graph training operation
The loss function (i.e. the value to minimize) is defined as the cross entropy between the predicted classes and the class ground truth. The train operation is then included within the graph as a weight/bias update operation.

Try changing the learning rate, how would setting a low or high learning rate affect the "fitting" to the training set ?

In [ ]:
# learning parameters
learning_rate = 1e-5
training_epochs = 500

# display training accuracy every ..
display_accuracy_step = 10

    
training_images, training_labels, test_images, test_labels = get_data()
logits, X, Y = multilayer_perceptron()

# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# define training graph operation
train_op = optimizer.minimize(loss_op)

# graph operation to initialize all variables
init_op = tf.global_variables_initializer()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  del sys.path[0]
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Run graph for specified number of epochs.

After the graph is defined, different operations in the graph can be run by specifying them in the sess.run() function.
A session is wrapper for running graphs. Outputs can also be acquired from the graph by including them in the variable list of sess.run().

In [ ]:
with tf.Session() as sess:

        # run graph weights/biases initialization op
        sess.run(init_op)
        # begin training loop ..
        for epoch in range(training_epochs):
            # complete code below
            # run optimization operation (backprop) and cost operation (to get loss value)
            _, cost = sess.run([train_op, loss_op], feed_dict={X: training_images,
                                                               Y: training_labels})

            # Display logs per epoch step
            print("Epoch:", '%04d' % (epoch + 1), "cost={:.9f}".format(cost))
                
            if epoch % display_accuracy_step == 0:
                pred = tf.nn.softmax(logits)  # Apply softmax to logits
                correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))

                # calculate training accuracy
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                print("Accuracy: {:.3f}".format(accuracy.eval({X: training_images, Y: training_labels})))

        print("Optimization Finished!")

        # -- Define and run test operation -- #
        
        # apply softmax to output logits
        pred = tf.nn.softmax(logits)
        
        #  derive inffered calasses as the class with the top value in the output density function
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        
        # calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            
        # complete code below
        # run test accuracy operation ..
        print("Test Accuracy:", accuracy.eval({X: test_images, Y: test_labels}))



Epoch: 0001 cost=0.695002615
Accuracy: 0.520
Epoch: 0002 cost=0.692600727
Epoch: 0003 cost=0.691955388
Epoch: 0004 cost=0.692250371
Epoch: 0005 cost=0.692385733
Epoch: 0006 cost=0.692014456
Epoch: 0007 cost=0.691375375
Epoch: 0008 cost=0.690776348
Epoch: 0009 cost=0.690397859
Epoch: 0010 cost=0.690240622
Epoch: 0011 cost=0.690162063
Accuracy: 0.520
Epoch: 0012 cost=0.689997196
Epoch: 0013 cost=0.689676762
Epoch: 0014 cost=0.689244986
Epoch: 0015 cost=0.688800454
Epoch: 0016 cost=0.688426375
Epoch: 0017 cost=0.688143253
Epoch: 0018 cost=0.687904537
Epoch: 0019 cost=0.687637806
Epoch: 0020 cost=0.687299430
Epoch: 0021 cost=0.686902642
Accuracy: 0.530
Epoch: 0022 cost=0.686499417
Epoch: 0023 cost=0.686137080
Epoch: 0024 cost=0.685822546
Epoch: 0025 cost=0.685518980
Epoch: 0026 cost=0.685180485
Epoch: 0027 cost=0.684790730
Epoch: 0028 cost=0.684371948
Epoch: 0029 cost=0.683958530
Epoch: 0030 cost=0.683564663
Epoch: 0031 cost=0.683175445
Accuracy: 0.600
Epoch: 0032 cost=0.682766676
Epoch: 0